<img src="./images/output-0.png" />

<img src="./images/output-1.png" />

<img src="./images/output-2.png" />

<img src="./images/output-4.png" />

<img src="./images/output-5.png" />

<img src="./images/output-6.png" />

<img src="./images/output-7.png" />

<img src="./images/output-8.png" />

<img src="./images/output-9.png" />

<img src="./images/output-10.png" />

<img src="./images/output-11.png" />

<img src="./images/output-12.png" />

<img src="./images/output-13.png" />

In [1]:
# Example One, getting frames off a webcam.
# basic_webcam.py
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

while(True):
    ret, frame = cap.read()
    cv2.imshow('Basic Web Cam',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [3]:
# Example two, advance camera properties. 
import numpy as np
import cv2
import time

pass 
# create a map to keep track of all these names
prop_map = {
    "pos_msec":cv2.cv.CV_CAP_PROP_POS_MSEC,
    "pos_frame":cv2.cv.CV_CAP_PROP_POS_FRAMES,
    "avi_ratio":cv2.cv.CV_CAP_PROP_POS_AVI_RATIO ,
    "width":cv2.cv.CV_CAP_PROP_FRAME_WIDTH ,
    "height":cv2.cv.CV_CAP_PROP_FRAME_HEIGHT ,
    "fps":cv2.cv.CV_CAP_PROP_FPS ,
    "fourcc":cv2.cv.CV_CAP_PROP_FOURCC ,
    "frame_count":cv2.cv.CV_CAP_PROP_FRAME_COUNT,
    "format":cv2.cv.CV_CAP_PROP_FORMAT ,
    "mode":cv2.cv.CV_CAP_PROP_MODE ,
    "brightness":cv2.cv.CV_CAP_PROP_BRIGHTNESS ,
    "contrast":cv2.cv.CV_CAP_PROP_CONTRAST ,
    "saturation":cv2.cv.CV_CAP_PROP_SATURATION,
    "hue":cv2.cv.CV_CAP_PROP_HUE ,
    "gain":cv2.cv.CV_CAP_PROP_GAIN ,
    "exposure":cv2.cv.CV_CAP_PROP_EXPOSURE ,
    "convert_rgb":cv2.cv.CV_CAP_PROP_CONVERT_RGB ,
 #   "white_balance":cv2.cv.CV_CAP_PROP_WHITE_BALANCE ,
    "rectification":cv2.cv.CV_CAP_PROP_RECTIFICATION}

# get a camera property
def get_prop(cam,name,prop_map):
    return cam.get(prop_map[name])

# set a camera property
def set_prop(cam,name,prop_map,value):
    cam.set(prop_map[name],value)

# print out all of the properites
def poll_props(cam,prop_map):
    out_map = {}
    for k,v in prop_map.items():
        result = cam.get(v)
        if( result == -1.0 ):
            out_map[k] = None
        else:
            out_map[k] = result
    return out_map

# create a camera and get its property
cam = cv2.VideoCapture(0)
properties = poll_props(cam,prop_map)

# list our properties
for k,v in properties.items():
    print "{0:<12}\t:{1:<12}".format(k,v)

while(True):
    # toggle properties and get results. 
    sat = get_prop(cam,"saturation",prop_map)
    if( sat > 0.5 ):
        set_prop(cam,"saturation",prop_map,0.1)
    else:
        set_prop(cam,"saturation",prop_map,1.0)
    time.sleep(0.05)
    ret, frame = cam.read()
    cv2.imshow('Basic Web Cam',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# put our toys back on the shelf 
cam.release()
cv2.destroyAllWindows()

hue         	:0.5         
saturation  	:0.10000000149
format      	:None        
brightness  	:0.6171875   
pos_frame   	:None        
convert_rgb 	:None        
rectification	:None        
height      	:480.0       
contrast    	:0.315789461136
width       	:640.0       
pos_msec    	:None        
fps         	:None        
fourcc      	:None        
mode        	:None        
avi_ratio   	:None        
frame_count 	:None        
gain        	:None        
exposure    	:None        


KeyboardInterrupt: 

<h1 align=center> Go Pro Cameras</h1>
<img src="/files/images/gopro.jpg" align=center>
<h2>
<ul align=left>
<li>All GoPro Cameras through 3+ use a similar interface.</li>
<br>
<li>Each camera acts as a wifi access point with a simple web server.</li>
<br>
<li>We can hack this interface for nefarious purposes.</li>
<br>
<li>Default path is http://10.5.5.9:8080/</li>
<br>
<li><a href="http://goprohero.readthedocs.org/en/latest/">GoProHero</a> is a nice library wrapping a lot of commands.</li>
</h2>
</ul>



<h1 align=center>Some Specifics on GoPros</h1>
<img src="/files/images/gopro_screen.png" align=center>
<h2>
<ul align=left>
<li>Interface is hitting specific routes with parameters.</li>
<br>
<li>Images and video are collected using urllib and beautiful soup.</li>
<br>
<li>Naming conventions are a little convoluted -- need some helper code.</li>
<br>
<li>Control over resolutions and frame rates. Gain and exposure not so much.</li>
<br>
<li>Live streaming is a bit wonky.</li>
</h2>
</ul>

In [1]:
from goprohero import GoProHero # helper lib
import urllib # used to "wget" files
import bs4 # beautiful soup, to parse file
import time
from IPython.display import Image

def download_and_save(name,route="http://10.5.5.9:8080/videos/DCIM/100GOPRO/"):
    """
    Download a specific file on a connected GoPro camera.
    """
    grab = route+name
    result = urllib.urlopen(grab) # basically wget
    if( result.code == 200 ): # 200 means OK
        with open(name,'wb') as fp: 
            fp.write(result.read()) # write to file
    result.close()

def get_new(last=[],url="http://10.5.5.9:8080/videos/DCIM/100GOPRO/"):
    """
    Find files recently added to our go pro. Requires a list of files
    """
    unique = None
    last = set(last)
    out = urllib.urlopen(url)
    if(out.code == 200):
        soup = bs4.BeautifulSoup(out.read()) # read the file tree
        # make a set of all <a href> in the tree
        fresh = set([row.renderContents() for row in soup.findAll('a')]) 
        # do a set difference with our initial set
        unique = list(fresh.difference(last))
    return unique



In [5]:
# check the camera file system
out = get_new()
# setup our controller
cam = GoProHero(password="herpderp")
# put the camera in burst mode
cam.command('mode','burst')
# tell it to take a shoth
cam.command('record','on')
# wait to finish
time.sleep(3)
# get the new files
out = get_new(last=out)
for o in out:
    print o
    download_and_save(o)

KeyboardInterrupt: 

In [ ]:
Image(out[0])


<h1 align=center>A lot of cameras work just like our GoPro</h1>
<img src="/files/images/phone_interface.png" align=center>
<h2>
<ul align=left>
<li>Most newer CCTV cameras, baby monitors, etc. are IP web cameras.</li>
<br>
<li>Each camera acts as a little web server and streams images.</li>
<br>
<li>Even better, every one of us has one in our pocket!</li>
<br>
<li>A large variety of Android/IOS apps as cameras.</li>
<br>
<li>Harder to write control interface, but generally easy to stream.</li>
</h2>
</ul>

<h1 align=center>A lot of cameras work just like our GoPro</h1>
<img src="/files/images/web_interface.png" align=center>
<h2>
<ul align=left>
<li>Most of these apps support <a href="">motion jpeg</a>.</li>
<br>
<li>MJPEG simply dumps a series of jpegs, i.e. only *intra* frame compression.</li>
<br>
<li>The trick is to find the begining and end of each file.</li>
<br>
<li>Ususally of very low quality to support streaming.</li>
<br>
<li>There are <a href="http://www.insecam.org/en/view/279373/">lots of them on the web.</a></li>
</h2>
</ul>

In [17]:
"""
Read and display a simple MJPEG stream given a URL
""" 
import cv2
import urllib 
import numpy as np
# open our url stream
stream=urllib.urlopen('http://192.168.1.228:8080/video')
# create a buffer of bytes
bytes=''
invert = False
while True:
    # read some bytes from the stream
    # how much to read is a matter of taste
    bytes+=stream.read(1024*8)
    # \xff\xd8 is the start of a jpg
    # \xff\xd9 is the end of a jpg
    a = bytes.find('\xff\xd8')
    b = bytes.find('\xff\xd9')
    # if we find them
    if a!=-1 and b!=-1:
        jpg = bytes[a:b+2] # create a buffer
        bytes= bytes[b+2:] # save the remaining bytes
        #numpy can handle strings as image data
        img = cv2.imdecode(np.fromstring(jpg, dtype=np.uint8),cv2.CV_LOAD_IMAGE_COLOR)
        # Add some bling to show we can process
        if invert:
            img = 255-img
        # show our image
        cv2.imshow("live_and_direct",img)
    # MVP keyboard input
    my_key = cv2.waitKey(1)
    if my_key & 0xFF == ord('q'):
        break
    if my_key & 0xFF == ord('i'):
        invert = not invert 
# put our toys away when done. 
stream.close()
cv2.destroyAllWindows()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 

In [15]:
cv2.imwrite?